In [0]:
!pip install transformers

     |████████████████████████████████| 317kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 33.0MB/s 
     |████████████████████████████████| 860kB 35.4MB/s 
     |████████████████████████████████| 645kB 48.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=d39efb3052312c9ea1d971b5482e226e62c9e96424954a64b0d319cd574112cd
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import sys
import pickle
import torch
import numpy as np
import torch.optim as optim
from transformers import BertConfig, BertModel, BertForMaskedLM

from IPython.display import clear_output


# import matplotlib.pyplot as plt
# from transformers import *

In [0]:
d = "drive/My Drive/Colab Notebooks/smaug/data"
p = os.path.join(d, "CDS_singlestrain_threeprime.pkl")
print(os.listdir(d))

['CDS_singlestrain_threeprime.pkl']


In [0]:
with open(p, 'rb') as f:
  CDS = pickle.load(f)

In [0]:
print(type(CDS))
print(len(CDS))
print(CDS[100])

<class 'list'>
9304394
HVSLLSVHAAVAVWRKKRQMYIDQYCVRGTKLTNAEKFVFTMYATVVGIKEDLMRVDASDINVSLIEQRRLNRIVDRRTMKNGDPSILIFDNAFIRELTVSDKSPYSRIWDENMRKLTEVSKNQAHLQCTFVMVLSFLLIAKM


In [0]:
lengths = [len(s) for s in CDS]
meanlen = np.mean(lengths)
medlen = np.median(lengths)
minlen = np.min(lengths)
maxlen = np.max(lengths)
print(meanlen, medlen, minlen, maxlen)

345.9973719943502 302.0 11 39686


In [0]:
# import logging
# logging.basicConfig(level=logging.INFO)

In [0]:
# # custom parameters for BERT model
# vocab_size = 12 # Vocabulary size of inputs_ids in BertModel. default=30522
# hidden_size = 768 # Size of the encoder layers and the pooler layer, default=768
# num_hidden_layers = 12 # Number of hidden layers in the Transformer encoder. default=12
# num_attention_heads = 12 # Number of attention heads for each attention layer in the Transformer encoder, default=12
# intermediate_size = 3072 # The size of the “intermediate” (i.e., feed-forward) layer in the Transformer encoder. default=3072
# hidden_act = "gelu" # The non-linear activation function (function or string) in the encoder and pooler. If string, “gelu”, “relu”, “swish” and “gelu_new” are supported. default="gelu"
# hidden_dropout_prob = 0.1 # The dropout probabilitiy for all fully connected layers in the embeddings, encoder, and pooler. default=0.1
# attention_probs_dropout_prob = 0.1 # The dropout ratio for the attention probabilities. default=0.1
# max_position_embeddings = 512 # The maximum sequence length that this model might ever be used with. Typically set this to something large just in case (e.g., 512 or 1024 or 2048). default=512
# type_vocab_size = 2 # 1 # The vocabulary size of the token_type_ids passed into BertModel. default=2
# initializer_range = 0.02 # The sttdev of the truncated_normal_initializer for initializing all weight matrices. default=0.02
# layer_norm_eps = 1e-12 # The epsilon used by LayerNorm. default=1e-12


# config = BertConfig(vocab_size_or_config_json_file=vocab_size,
#                     hidden_size=hidden_size,
#                     num_hidden_layers=num_hidden_layers,
#                     num_attention_heads=num_attention_heads,
#                     intermediate_size=intermediate_size,
#                     hidden_act=hidden_act,
#                     hidden_dropout_prob=hidden_dropout_prob,
#                     attention_probs_dropout_prob=attention_probs_dropout_prob,
#                     max_position_embeddings=max_position_embeddings,
#                     type_vocab_size=type_vocab_size,
#                     initializer_range=initializer_range,
#                     layer_norm_eps=layer_norm_eps)

# model = BertForMaskedLM(config)

# print(model)
# model.to('cuda')

In [0]:
# custom parameters for BERT model
vocab_size = 22 # Vocabulary size of inputs_ids in BertModel. default=30522
hidden_size = 40 # Size of the encoder layers and the pooler layer, default=768
num_hidden_layers = 2 # Number of hidden layers in the Transformer encoder. default=12
num_attention_heads = 2 # Number of attention heads for each attention layer in the Transformer encoder, default=12
intermediate_size = hidden_size*4 # The size of the “intermediate” (i.e., feed-forward) layer in the Transformer encoder. default=3072
hidden_act = "gelu" # The non-linear activation function (function or string) in the encoder and pooler. If string, “gelu”, “relu”, “swish” and “gelu_new” are supported. default="gelu"
hidden_dropout_prob = 0.1 # The dropout probabilitiy for all fully connected layers in the embeddings, encoder, and pooler. default=0.1
attention_probs_dropout_prob = 0.1 # The dropout ratio for the attention probabilities. default=0.1
max_position_embeddings = 50 # The maximum sequence length that this model might ever be used with. Typically set this to something large just in case (e.g., 512 or 1024 or 2048). default=512
type_vocab_size = 1 # 1 # The vocabulary size of the token_type_ids passed into BertModel. default=2
initializer_range = 0.02 # The sttdev of the truncated_normal_initializer for initializing all weight matrices. default=0.02
layer_norm_eps = 1e-12 # The epsilon used by LayerNorm. default=1e-12


config = BertConfig(vocab_size_or_config_json_file=vocab_size,
                    hidden_size=hidden_size,
                    num_hidden_layers=num_hidden_layers,
                    num_attention_heads=num_attention_heads,
                    intermediate_size=intermediate_size,
                    hidden_act=hidden_act,
                    hidden_dropout_prob=hidden_dropout_prob,
                    attention_probs_dropout_prob=attention_probs_dropout_prob,
                    max_position_embeddings=max_position_embeddings,
                    type_vocab_size=type_vocab_size,
                    initializer_range=initializer_range,
                    layer_norm_eps=layer_norm_eps)

model = BertForMaskedLM(config)

print(model)
model.to('cuda')

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(22, 40, padding_idx=0)
      (position_embeddings): Embedding(50, 40)
      (token_type_embeddings): Embedding(1, 40)
      (LayerNorm): LayerNorm((40,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=40, out_features=40, bias=True)
              (key): Linear(in_features=40, out_features=40, bias=True)
              (value): Linear(in_features=40, out_features=40, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=40, out_features=40, bias=True)
              (LayerNorm): LayerNorm((40,), eps=1e-12, elementwise_affine=True)
             

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(22, 40, padding_idx=0)
      (position_embeddings): Embedding(50, 40)
      (token_type_embeddings): Embedding(1, 40)
      (LayerNorm): LayerNorm((40,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=40, out_features=40, bias=True)
              (key): Linear(in_features=40, out_features=40, bias=True)
              (value): Linear(in_features=40, out_features=40, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=40, out_features=40, bias=True)
              (LayerNorm): LayerNorm((40,), eps=1e-12, elementwise_affine=True)
             

In [0]:
# encode data as GPU tensors
max_aa_seq_length = max_position_embeddings


def tokenize_aa_seq_murphy10(aa_seq):
    table = {"L":1,
             "V":1,
             "I":1,
             "M":1,
             "C":2,
             "A":3,
             "G":4,
             "S":5,
             "T":5,
             "P":6,
             "F":7,
             "Y":7,
             "W":7,
             "E":8,
             "D":8,
             "N":8,
             "Q":8,
             "K":9,
             "R":9,
             "H":10,
             "X":0, # get rid of those
             "B":0,
             "*":0}
    tokenized = [table[aa] for aa in aa_seq]
    return tokenized

def tokenize_aa_seq(aa_seq):
    table = {"L":1,
             "V":2,
             "I":3,
             "M":4,
             "C":5,
             "A":6,
             "G":7,
             "S":8,
             "T":9,
             "P":10,
             "F":11,
             "Y":12,
             "W":13,
             "E":14,
             "D":15,
             "N":16,
             "Q":17,
             "K":18,
             "R":19,
             "H":20,
             "X":0, # get rid of those
             "B":0,
             "*":0}
    tokenized = [table[aa] for aa in aa_seq]
    return tokenized

# tokens = [tokenize_aa_seq_murphy10(seq) for seq in CDS[:1000]] ######################################
tokens = [tokenize_aa_seq(seq) for seq in CDS[:1000]] ######################################

tokens_tensor = torch.zeros(len(tokens), max_aa_seq_length, dtype=torch.long)
for i in range(len(tokens)):
    l = len(tokens[i]) # scuff way to ensure fit in tensor, TODO build correctly sized data set and split into train test
    if l > max_aa_seq_length:
        l = max_aa_seq_length
    for j in range(l):
        tokens_tensor[i][j] += tokens[i][j]

tokens_tensor = tokens_tensor.to('cuda')

print(tokens_tensor)
print(tokens_tensor.shape)

tensor([[18,  1,  4,  ...,  9,  6,  7],
        [ 6, 17,  6,  ...,  4,  2, 11],
        [15, 19,  1,  ..., 18,  8,  9],
        ...,
        [ 1, 15, 14,  ...,  2,  8, 15],
        [18,  1, 18,  ...,  1, 17,  5],
        [ 3, 19,  6,  ...,  2,  6, 13]], device='cuda:0')
torch.Size([1000, 50])


In [0]:
dtrain = tokens_tensor[:int(0.8*len(tokens_tensor))] #TODO do an actual random selection on better data
dvalid = tokens_tensor[int(0.8*len(tokens_tensor)):]
print(len(dtrain))
print(len(dvalid))

800
200


In [0]:
# optimizer = optim.SGD(model.parameters(), lr=0.0001)#, momentum=args.momentum)
optimizer = optim.AdamW(model.parameters())
# optimizer = optim.Adam(model.parameters())

In [0]:
# train model with single aa masked at a time ##### batch
model.train()

batch_size = 2**12


loss_mask = (torch.zeros(batch_size, max_position_embeddings)-1).long().to('cuda')

np.random.seed(2019)
np.random.seed(10)

model.zero_grad()
optimizer.zero_grad()

for i in range(1000):
    select_idx = np.random.randint(0, len(dtrain), batch_size)
#     input_ids = dtrain[select_idx].unsqueeze(0)
    input_ids_correct = dtrain[select_idx]
    input_ids = input_ids_correct.clone()

    for j in range(max_position_embeddings):
        tempval = torch.zeros(batch_size).long()

        for k in range(batch_size):
            tempval[k] = input_ids[k, j].item()
        
        optimizer.zero_grad()
        model.zero_grad()

        loss_mask[:, j] = tempval # calculate loss based only on masked amino acid
      
        input_ids[:, j] = vocab_size-1 # mask label is the highest vocab number, never present in data

        outputs = model(input_ids, masked_lm_labels=loss_mask) #TODO mask padding 0's for attention
#         outputs = model(input_ids, masked_lm_labels=input_ids_correct) #TODO mask padding 0's for attention
#         print(outputs)

        loss, prediction_scores = outputs[:2]
#         loss, prediction_scores = outputs

        
        # for k in range(batch_size):
        clear_output(wait=True)
        for k in range(3):
            print(i, "\t", torch.argmax(prediction_scores[k,j]).item(), '\t', loss_mask[k,j].item(), "\t", loss.item())
            print(prediction_scores[k,j])
        print()
        
        loss.backward()
        optimizer.step()

        for k in range(batch_size):
            input_ids[k, j] = tempval[k] # set masked value back to original value
            loss_mask[k, j] = -1

#         else:
#             break # stop training at end of protein sequence

#     break

3 1 	 12 	 2.858600616455078
tensor([-2.4624,  1.2467,  0.7528,  0.8444, -0.2993, -1.5819,  1.0521,  0.8769,
         0.6866,  0.6325,  0.3416,  0.5186,  0.0666, -0.6995,  0.7221,  0.4601,
         0.2751,  0.0170,  1.0462,  0.5948, -0.3550, -2.3411], device='cuda:0',
       grad_fn=<SelectBackward>)
3 1 	 12 	 2.858600616455078
tensor([-2.6119,  1.2514,  0.8257,  0.8974, -0.2989, -1.6216,  1.0534,  0.8919,
         0.6413,  0.6600,  0.4567,  0.5320,  0.1353, -0.7860,  0.7546,  0.4764,
         0.2847,  0.1360,  1.1082,  0.5964, -0.3567, -2.3738], device='cuda:0',
       grad_fn=<SelectBackward>)
3 1 	 3 	 2.858600616455078
tensor([-2.3881,  1.1734,  0.7367,  0.8986, -0.2904, -1.5358,  1.0004,  0.7635,
         0.5814,  0.5518,  0.4343,  0.5683,  0.1160, -0.7785,  0.7463,  0.4270,
         0.2483,  0.0958,  1.0449,  0.5569, -0.2832, -2.2015], device='cuda:0',
       grad_fn=<SelectBackward>)



KeyboardInterrupt: ignored

In [0]:
# TRIVIAL TEST, TRAIN A MODEL ON SAME DATA REPEATEDLY
model.train()

batch_size = 2**10


loss_mask = (torch.zeros(batch_size, max_position_embeddings)-1).long().to('cuda')

np.random.seed(2019)
for i in range(1000):
#     select_idx = np.random.randint(0, len(dtrain), batch_size)
    select_idx = np.random.randint(0, 1, batch_size)

#     input_ids = dtrain[select_idx].unsqueeze(0)
    input_ids_correct = dtrain[select_idx]
    input_ids = input_ids_correct.clone()

    for j in range(max_position_embeddings):
        tempval = torch.zeros(batch_size).long()

        for k in range(batch_size):
            tempval[k] = input_ids[k, j].item()
        
        optimizer.zero_grad()

        loss_mask[:, j] = tempval # calculate loss based only on masked amino acid
      
        input_ids[:, j] = vocab_size-1 # mask label is the highest vocab number, never present in data

#         outputs = model(input_ids, masked_lm_labels=input_ids_correct) #TODO mask padding 0's for attention
        outputs = model(input_ids, masked_lm_labels=loss_mask) #TODO mask padding 0's for attention
        print(loss_mask.shape)
#         print(input_ids)
        sys.exit()

        loss, prediction_scores = outputs[:2]
#         loss, prediction_scores = outputs

        clear_output(wait=True)
        for k in range(3):
            print(i, torch.argmax(prediction_scores[k+1,j]).item(), loss_mask[k,j].item(), "\t", loss.item())
        
        print()
        
        loss.backward()
        optimizer.step()

        for k in range(batch_size):
            input_ids[k, j] = tempval[k] # set masked value back to original value
            loss_mask[k, j] = -1

#         else:
#             break # stop training at end of protein sequence

#     break

torch.Size([1024, 50])


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
# evaluate the model using softmax proportion of correct label
model.eval()

np.random.seed(424242)

maxsum = 0
softsum = 0
n_correct = 0
n_total = 0
for i in range(100):
    select_idx = np.random.randint(0,len(dvalid),1)[0]    
    input_ids = dvalid[select_idx].unsqueeze(0)
    # print(input_ids)
    
    for j in range(max_position_embeddings):
        tempval = 0
        tempval += input_ids[0, j].item()
        if tempval != 0:
            input_ids[0, j] = vocab_size-1 # mask label is the highest vocab number, never present in data
            logits = model(input_ids)[0][0,j]
            soft = torch.exp(logits)
            softproportion = (soft[tempval]/torch.sum(soft)).item() #######bug tempval not j
            softsum += softproportion
            maxsum += 1

            pred = torch.argmax(logits).item()
            actual = tempval
            n_correct += pred==actual
            n_total +=1
            
            input_ids[0,j] = tempval # set masked value back to original value

correctness = softsum/maxsum
print(correctness)

correctness_p = n_correct/n_total
print(correctness_p)

0.06227649919032701
0.0844


In [0]:
# evaluate the model using softmax proportion of correct label, ON NOISE
model.eval()

np.random.seed(424242)

maxsum = 0
softsum = 0
n_correct = 0
n_total = 0
for i in range(100):
#     select_idx = np.random.randint(0,len(dvalid),1)[0]    
#     input_ids = dvalid[select_idx].unsqueeze(0)
    noise = np.random.randint(1, 20, max_position_embeddings).reshape(1,-1)
    input_ids = torch.from_numpy(noise).to('cuda')
#     print(input_ids)
    
    for j in range(max_position_embeddings):
        tempval = 0
        tempval += input_ids[0, j].item()
        if tempval != 0:
            input_ids[0, j] = vocab_size-1 # mask label is the highest vocab number, never present in data
            logits = model(input_ids)[0][0,j]
            soft = torch.exp(logits)
            softproportion = (soft[tempval]/torch.sum(soft)).item()
            softsum += softproportion
            maxsum += 1
#             print(softproportion)

            pred = torch.argmax(logits).item()
            actual = tempval
            n_correct += pred==actual
            n_total +=1
            
            input_ids[0,j] = tempval # set masked value back to original value

correctness = softsum/maxsum
print(correctness)

correctness_p = n_correct/n_total
print(correctness_p)

KeyboardInterrupt: ignored

In [0]:
# evaluate using model that just predicts most common amino acid
model.eval()

np.random.seed(424242)


maxsum = 0
softsum = 0
n_correct = 0
n_total = 0
for i in range(100):
    select_idx = np.random.randint(0,len(dvalid),1)[0]    
    input_ids = dvalid[select_idx].unsqueeze(0)
    
    for j in range(max_position_embeddings):
        tempval = 0
        tempval += input_ids[0, j].item()
        if tempval != 0:
            input_ids[0, j] = vocab_size-1 # mask label is the highest vocab number, never present in data
            soft = torch.zeros(vocab_size)
            soft[1] += 1
            softproportion = (soft[tempval]/torch.sum(soft)).item()
            softsum += softproportion
            maxsum += 1

            pred = torch.argmax(soft).item()

            actual = tempval
            n_correct += pred==actual
            n_total += 1
            
            input_ids[0,j] = tempval # set masked value back to original value

correctness = softsum/maxsum
print(correctness)

correctness_p = n_correct/n_total
print(correctness_p)

0.0878
0.0878


In [0]:
# train model
batch_size = 1

optimizer = optim.SGD(model.parameters(), lr=0.01)#, momentum=args.momentum)
# optimizer = optim.AdamW(model.parameters())
_max_select = len(dtrain)


# single step to non-zero weights
optimizer.zero_grad()
select_idx = np.random.randint(0, _max_select, batch_size)
outputs = model(dtrain[select_idx], masked_lm_labels=dtrain[select_idx])
loss, prediction_scores = outputs[:2]
loss.backward()
optimizer.step()

# # evaluate before training
# acc_train = evaluate(model, dtrain)
# acc_valid = evaluate(model, dvalid)
# print(acc_train)
# print(acc_valid)


# train
model.train()
np.random.seed(2019)
for i in range(100):
    optimizer.zero_grad()
    select_idx = np.random.randint(0, _max_select, batch_size)
    input_ids = tokens_tensor[select_idx]
    outputs = model(input_ids, masked_lm_labels=input_ids)
    loss, prediction_scores = outputs[:2]
    print(i, loss.item())
    loss.backward()
    optimizer.step()
    
#     loss.backward(retain_graph=True)



In [0]:
# after training
acc_train = evaluate(model, dtrain)
acc_valid = evaluate(model, dvalid)
print(acc_train)
print(acc_valid)


In [0]:
model.eval()
outputs = model(dvalid[0].unsqueeze(0), masked_lm_labels=dvalid[0].unsqueeze(0))
loss, prediction_scores = outputs[:2]

predicted_index = torch.argmax(prediction_scores, dim=2)
print(predicted_index)
print(dvalid[0])

In [0]:
# debug
model.train()
optimizer = optim.SGD(model.parameters(), lr=0.01)#, momentum=args.momentum)
for i in range(10):
    optimizer.zero_grad()
#     select_idx = 1
#     input_ids = tokens_tensor[select_idx].unsqueeze(0)
#     input_ids = tokens_tensor[0:3]
    input_ids = dtrain[0].unsqueeze(0)

    outputs = model(input_ids, masked_lm_labels=input_ids)
    loss, prediction_scores = outputs[:2]
    print(i, loss.item())
    loss.backward()
    optimizer.step()
    
#     loss.backward(retain_graph=True)



In [0]:
model.eval()
select_idx = 5
input_ids = tokens_tensor[select_idx].unsqueeze(0)
outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]

predicted_index = torch.argmax(prediction_scores, dim=2)
a = evaluate(model, tokens_tensor[select_idx].unsqueeze(0))
print(a)
# print(predicted_index.shape)
print(predicted_index)

In [0]:
# evaluate on random data to ensure accuracy metric works
model.eval()

np.random.seed(2019)
noise = np.random.randint(1, 10, 512).reshape(1,-1)
input_ids = torch.from_numpy(noise).to('cuda')

outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]

predicted_index = torch.argmax(prediction_scores, dim=2)
a = evaluate(model, tokens_tensor[select_idx].unsqueeze(0))
print(a)
# print(predicted_index.shape)
print(input_ids)
print(predicted_index)

In [0]:
# fix forward mask
model.eval()

np.random.seed(2019)
noise = np.random.randint(1, 10, 512).reshape(1,-1)
input_ids = torch.from_numpy(noise).to('cuda')
input_ids[0,0] = -inf

attention_mask = torch.from_numpy(np.ones(512).reshape(1,-1)).float().to('cuda')
attention_mask[0,0] = 0
# print(attention_mask)

token_type_ids = torch.from_numpy(np.zeros(512).reshape(1,-1)).long().to('cuda')

outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]

predicted_index = torch.argmax(prediction_scores, dim=2)
a = evaluate(model, tokens_tensor[select_idx].unsqueeze(0))
print(a)
# print(predicted_index.shape)
print(input_ids)
print(predicted_index)

In [0]:
del model
torch.cuda.empty_cache()

In [0]:
# train model
import torch.optim as optim
import torch.nn.functional as F

def train_epoch(epoch, args, model, device, data_loader, optimizer):
    model.train()  # set to training mode, disappointingly does not actually train the model 
    pid = os.getpid()
    for batch_idx, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = F.nll_loss(output, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('{}\tTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                pid, epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item()))
            
optimizer = optim.SGD(model.parameters(), lr=0.01)#, momentum=args.momentum)
optimizer.zero_grad()
# output = model(tokens_tensor)
# loss = F.nll_loss(output, target.to('cuda'))
print([x for x in model.parameters()])


In [0]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model = BertForMaskedLM(config)

input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]
print(loss)
print(prediction_scores)

In [0]:
print(input_ids.shape)
print(tokens_tensor[0].unsqueeze(0).shape)
print(tokens_tensor[0:2].shape)

In [0]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model = BertForMaskedLM(config)
model.to('cuda')

# input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
# input_ids = tokens_tensor[0].unsqueeze(0)
input_ids = tokens_tensor[0:2]

outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]
print(loss)
print(prediction_scores.shape)

for i in range(2):
    input_ids = tokens_tensor[0:2]
    outputs = model(input_ids, masked_lm_labels=input_ids)
    loss, prediction_scores = outputs[:2]
    loss.backward()
    print(loss)
    print(prediction_scores.shape)
#     loss.backward(retain_graph=True)

In [0]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
# import logging
# logging.basicConfig(level=logging.INFO)
# logging.basicConfig(level=logging.NONE)


# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
# masked_index = 8
# tokenized_text[masked_index] = '[MASK]'

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [0]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
# segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
#     outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    outputs = model(tokens_tensor)

    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

predicted_index = torch.argmax(predictions[0, 11]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

In [0]:
for i in range(len(indexed_tokens)):
    predicted_index = torch.argmax(predictions[0, i]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    print(predicted_token)

In [0]:
predicted_token = tokenizer.convert_ids_to_tokens([103])[0]
print(predicted_token)

In [0]:
import logging
logging.basicConfig(level=logging.INFO)

In [0]:
# example tokenization
from transformers import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

masked_index = 8
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)
print(tokens_tensor.shape)

In [0]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
# segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
#     outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

In [0]:
# GPT-2

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2-large')


# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()
model.to('cuda')

100%|██████████| 3247202234/3247202234 [03:41<00:00, 14645396.05B/s]


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [0]:
# original_text = "Martin Steinegger is in Peru because he "
# original_text = "The real reason Steven keeps recruiting German postdocs is "
# original_text = "The secret to giving a fun and compelling Joint Lab Meeting presentation is "
# original_text = "UC Berkeley is"
# original_text = "Why did Donald Trump "
# original_text = "Finding genes is easy... The secret is "
# original_text = "Computational gene finding is easy, the problem is "
# original_text = "The future of Biomedical Engineering is "
# original_text = "The best way to describe how neural networks work is "
original_text = "Improving on state-of-the-art bacterial gene finding programs is hard, "


text = original_text
for i in range(20): # not the best way to iterate, but it works
    if text[-1] != ".":
        # Encode a text inputs
        indexed_tokens = tokenizer.encode(text)

        # Convert indexed tokens in a PyTorch tensor
        tokens_tensor = torch.tensor([indexed_tokens])

        # If you have a GPU, put everything on cuda
        tokens_tensor = tokens_tensor.to('cuda')

        # Predict all tokens
        with torch.no_grad():
            outputs = model(tokens_tensor)
            predictions = outputs[0]

        # get the predicted next sub-word (in our case, the word 'man')
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

        text = predicted_text

In [0]:
print("Original text:\t\t", original_text)
print("Completed sentence:\t", predicted_text)

Original text:		 Improving on state-of-the-art bacterial gene finding programs is hard, 
Completed sentence:	 Improving on state-of-the-art bacterial gene finding programs is hard, but it's not impossible.
